# PART 1: Simple Markowitz-Model implementation to estimate weightage of each stock in portfolio, where portfolio will have highest sharpe ratio


In [ ]:
### PART 1: Simple Markowitz-Model implementation to estimate weightage of each stock in portfolio, where portfolio will have highest sharpe ratio

# Step 1: Import modules and define constants
import numpy as np
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as optimization
from scipy import interpolate
from scipy import stats
import plotly.express as px
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


# On avge, 252 trading days a year
NUM_TRADING_DAYS = 252

# Generate random w (diff. portfolios)
NUM_PORTFOLIOS = 10000

# Our portfolio stocks
stocks = ['AAPL', 'NVDA', 'MAR', 'LMT', 'BA']

# Historical data - define START, END dates
start_date = '2010-01-01'
end_date = '2023-10-01'

In [ ]:
# Step 2: Data retrieval and Visualisation
def download_data():
    # name of stock as key, stock values 2010-2023
    stock_data = {}
    for stock in stocks:
        ticker = yf.Ticker(stock)
        stock_data[stock] = ticker.history(start=start_date, end=end_date)['Close']
    return pd.DataFrame(stock_data)

def show_data(data):
    data.plot(figsize=(10, 5))
    plt.show()

In [ ]:
# Step 3: Returns calculation and Portfolio Statistics
# To calculate return using normalisation, show statistics using annual metrics
def calculate_return(data):
    # Normalization - to measure variables in comparable metric
    log_return = np.log(data / data.shift(1))
    return log_return[1:]                    # Don't need first row of dataset, since values are NaN

def show_statistics(returns):
    # Instead of daily metrics, use annual metrics
    annual_mean = returns.mean() * NUM_TRADING_DAYS
    annual_covariance = returns.cov() * NUM_TRADING_DAYS
    print("Annual Mean Returns:")
    print(annual_mean)
    print("Annual Covariance Matrix:")
    print(annual_covariance)

In [ ]:
# Step 4: To visaulise efficient frontier and displau portfolio metrics
def show_mean_variance(returns, weights):
    portfolio_return = np.sum(returns.mean() * weights) * NUM_TRADING_DAYS
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * NUM_TRADING_DAYS, weights)))
    print("Expected portfolio mean (return): ", portfolio_return)
    print("Expected portfolio volatility (standard deviation): ", portfolio_volatility)


# Generate multiple portfolios: Display their efficient frontier
def show_portfolios(returns, volatilities):
    plt.figure(figsize=(10, 6))
    plt.scatter(volatilities, returns, c=returns/volatilities, marker='o')
    plt.grid(True)
    plt.xlabel('Expected Volatility')
    plt.ylabel('Expected Return')
    plt.colorbar(label='Sharpe Ratio')
    plt.show()

In [ ]:
# Step 5: Portfolio Generation
def generate_portfolios(returns):
    portfolio_means = []
    portfolio_risks = []
    portfolio_weights = []

    for _ in range(NUM_PORTFOLIOS):
        # Generate random weight
        w = np.random.random(len(stocks))
        w /= np.sum(w)
        portfolio_weights.append(w)
        portfolio_mean = np.sum(returns.mean() * w) * NUM_TRADING_DAYS
        portfolio_risk = np.sqrt(np.dot(w.T, np.dot(returns.cov() * NUM_TRADING_DAYS, w)))
        portfolio_means.append(portfolio_mean)
        portfolio_risks.append(portfolio_risk)

    return np.array(portfolio_weights), np.array(portfolio_means), np.array(portfolio_risks)


In [ ]:
# Step 6: Portfolio Optimization Functions
def statistics(weights, returns):
    portfolio_return = np.sum(returns.mean() * weights) * NUM_TRADING_DAYS
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * NUM_TRADING_DAYS, weights)))
    return np.array([portfolio_return, portfolio_volatility, portfolio_return / portfolio_volatility])

# Scipy optimize module can find the minimum of a given function
# Maximum of f(x) is minimum of -f(x)
def min_function_sharpe(weights, returns):
    return -statistics(weights, returns)[2]

# Constraints: Sum of weights = 1
# f(x) = 0 this is the function to minimize
def optimize_portfolio(weights, returns):
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    # Weights can be 1 at most: 1 when 100% of money invested in single stock
    bounds = tuple((0, 1) for _ in range(len(stocks)))
    return optimization.minimize(fun=min_function_sharpe, x0=weights[0], args=returns, method='SLSQP', bounds=bounds, constraints=constraints)


In [ ]:
# Step 7: Portfolio Results Presentation
def print_optimal_portfolio(optimum, returns):
    print("Optimal portfolio weights:", optimum['x'].round(3))
    expected_return, volatility, sharpe_ratio = statistics(optimum['x'].round(3), returns)
    print(f"Expected return: {expected_return}")
    print(f"Expected volatility (standard deviation): {volatility}")
    print(f"Sharpe ratio: {sharpe_ratio}")

def show_optimal_portfolio(opt, rets, portfolio_rets, portfolio_vols):
    plt.figure(figsize=(10, 6))
    plt.scatter(portfolio_vols, portfolio_rets, c=portfolio_rets / portfolio_vols, marker='o')
    plt.grid(True)
    plt.xlabel('Expected Volatility')
    plt.ylabel('Expected Return')
    plt.colorbar(label='Sharpe Ratio')
    plt.plot(statistics(opt['x'], rets)[1], statistics(opt['x'], rets)[0], 'g*', markersize=20.0)
    plt.show()

In [ ]:
# Main Workflow
# First output: Display stock prices of the 5 stocks, using yfinance
dataset = download_data()
show_data(dataset)
print('\n')

# Second output: Annual Mean returns and covariance matrix of stocks in portfolio
log_daily_returns = calculate_return(dataset)
show_statistics(log_daily_returns)
print('\n')

# Third output: Display Scatterplot of expected return against expected volatility, color coded by sharpe ratio
weights, means, risks = generate_portfolios(log_daily_returns)
show_portfolios(means, risks)
print('\n')

# Fourth output: Display most optimal portfolio (highest sharpe ratio), denoted by green star
# Each value in the list for optimal portfolio shows the corresponding optimal percentage each stock should take in portfolio
optimum = optimize_portfolio(weights, log_daily_returns)
print_optimal_portfolio(optimum, log_daily_returns)
show_optimal_portfolio(optimum, log_daily_returns, means, risks)

# Part 2: Incorporating sentiment score of earnings call and technical indicators with stock data

In [ ]:
### PART 2: Analysing of the 5 stock prices, with incorporation of their corresponding sentiment score (extracted using NLP and FinBert)
### NOTE: Wil not be forecasting prices because there are measured in absolute terms and thus harder to compare across time and other assets.
###       Hence, will be forecasting daily returns instead

# Step 1: Import modules and load data
import yfinance as yf
import math
from datetime import date
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
import seaborn as sns

In [ ]:
stocks = ['AAPL', 'NVDA', 'MAR', 'LMT', 'BA']
# stocks = []
stock_data = yf.download(stocks, start='2010-01-01', end='2023-10-20', group_by='tickers')
stock_data.head()

In [ ]:
# Step 2: Clean and explore data
stock_data.describe()

In [ ]:
# Check for presence of missing values
missing_values_count = stock_data.isnull().sum()
missing_values_count

In [ ]:
# Check shape of dataframe
stock_data.shape

In [ ]:
# Check columns of dataframe
stock_data.columns

In [ ]:
aapl = yf.Ticker('AAPL')

# uncomment later
# aapl.info

In [ ]:
# Explore data: Create line charts for columns of dataset
# Define the DataFrame to plot (Will display line charts for 5 stocks separately here for ease of visualisation)
aapl_df = yf.download('AAPL', start='2010-01-01', end='2023-10-20')
nvda_df = yf.download('NVDA', start='2010-01-01', end='2023-10-20')
mar_df = yf.download('MAR', start='2010-01-01', end='2023-10-20')
lmt_df = yf.download('LMT', start='2010-01-01', end='2023-10-20')
ba_df = yf.download('BA', start='2010-01-01', end='2023-10-20')

# Function to plot line chart of every stock in stock_df (For a quick glance)
def plot_line_charts(stock_df):
    # Plot line charts
    df_plot = stock_df.copy()

    ncols = 2
    nrows = int(round(df_plot.shape[1] / ncols, 0))

    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, sharex=True, figsize=(14, 7))
    for i, ax in enumerate(fig.axes):
        sns.lineplot(data = df_plot.iloc[:, i], ax=ax)
        ax.tick_params(axis="x", rotation=30, labelsize=10, length=0)
        ax.xaxis.set_major_locator(mdates.AutoDateLocator())
    fig.tight_layout()
    plt.show()

print("Line charts for AAPL: \n")
plot_line_charts(aapl_df)
print("Line charts for NVDA: \n")
plot_line_charts(nvda_df)
print("Line charts for MAR: \n")
plot_line_charts(mar_df)
print("Line charts for LMT: \n")
plot_line_charts(lmt_df)
print("Line charts for BA: \n")
plot_line_charts(ba_df)

In [ ]:
# Step 3: Feature Engineering
# Forecast daily returns by creating return column and analyse possible outliers using boxplot

for stock in stocks:
    stock_data[stock, 'Daily_Return'] = stock_data[stock]['Adj Close'].pct_change()
    stock_data = stock_data.sort_index(axis=1)
# print(stock_data)

# Perform data analysis to identify possible outliers
plt.figure(figsize=(12, 6))
for stock in stocks:
    sns.boxplot(x=stock_data[stock]["Daily_Return"])
    plt.title(f'Box Plot for {stock} Daily Returns')
    plt.show()
    print("\n")

In [ ]:
# Since we feel that outliers are meaningful and representative of actual market conditions/events, we decide not to remove them
# Using log transformation to reduce the impact of outliers, instead of removing them
# Will be log tansforming the outliers to certain features in dataframe merged

stock_data.head()

#Import Sentiment score from Excel

In [ ]:
# Adding text sentiment score as an additional column to dataframe

# Import excel data of sentiment score as dataframe
df_sentiment_score = pd.read_excel("sentiment_score.xlsx")
df_sentiment_score

In [ ]:
# Convert filename column, data will be in the format of YYYY-MM-DD.TickerName
def convert_column(filename):
    file_name_splitted = filename.split('.')
    final_file_name = ""
    month_to_numeric = {"Jan": "01",
                        "Feb": "02",
                        "Mar": "03",
                        "Apr": "04",
                        "May": "05",
                        "Jun": "06",
                        "Jul": "07",
                        "Aug": "08",
                        "Sep": "09",
                        "Oct": "10",
                        "Nov": "11",
                        "Dec": "12"
                        }

    splitted_by_dash = file_name_splitted[0].split('-')
    month_alphabet = splitted_by_dash[1]
    month_numeric = month_to_numeric[month_alphabet]
    final_file_name += splitted_by_dash[0] + "-" + month_numeric + "-" + splitted_by_dash[2] + "." + splitted_by_dash[3]

    return final_file_name

df_sentiment_score['filename'] = df_sentiment_score['filename'].apply(convert_column)
df_sentiment_score

In [ ]:
# Function to make the date as row index, while column will be stock ticker
def format_filename(row):
    parts = row['filename'].split('.')
    return pd.Series([parts[0], parts[1], row['sentiment_score']], index=['Date', 'Ticker', 'Sentiment_score'])

# Apply the function to each row and restructure the DataFrame
df_sentiment_score = df_sentiment_score.apply(format_filename, axis=1)

# Set the 'Date' column as the row index
df_sentiment_score.set_index('Date', inplace=True)
# df_sentiment_score.groupby("Ticker")


In [ ]:
df_sentiment_score = df_sentiment_score.groupby("Ticker")

sentiment_score_dict = {}
for stock in stocks:
    suffixed_df = df_sentiment_score.get_group(f"{stock}").add_prefix(f'{stock}_')
    suffixed_df.drop([f"{stock}_Ticker"], axis=1, inplace=True)
    new_df = suffixed_df.reset_index('Date')
    sentiment_score_dict[f'{stock}_df_sentiment_score'] = new_df

# print(sentiment_score_dict)


In [ ]:
sentiment_score_dict

In [ ]:
stock_data

In [ ]:
stock_data_dict = {}

for stock in stocks:
    # print(stock)
    suffixed_df = stock_data[f'{stock}'].add_prefix(f'{stock}_')
    new_df = suffixed_df.reset_index('Date')
    # print(new_df)
    new_df['Date'] = new_df['Date'].astype(str)
    stock_data_dict[f'{stock}_stock_data'] = new_df

# for key,value in stock_data_dict.items():
#     print(key)


In [ ]:
print(stock_data_dict)

# Merge sentiment_score dataframe with stock_data dataframe

In [ ]:
# aapl_merged = aapl.merge(aaplSentiment, how='outer', on='Date')

merged_dict = {}
for stock in stocks:
    new_merged_df = stock_data_dict[f'{stock}_stock_data'].merge(sentiment_score_dict[f'{stock}_df_sentiment_score'], how='outer', on='Date')
    merged_dict[f"{stock}"] = new_merged_df

# print(merged_dict)


In [ ]:
res = list(merged_dict.keys())[0]

merged = pd.DataFrame(merged_dict[res])
print(merged)

for key, value in merged_dict.items():
    if key != res:
        # print(value)
        merged = merged.merge(value, on='Date', how='outer')

merged = merged.sort_values(by=['Date'], ascending=True)
print(merged)
# merged.to_excel('merged.xlsx', index=False)

In [ ]:
# merged is a dataframe that has 1 additional row than dataframe stock_data because an earnings call by MAR was done on a Saturday (2019-05-11)
merged.head()

# Data Cleaning, Exploratory Data Analysis and Feature Engineering

In [ ]:
# I Will want to "data clean"/handle outliers for initial columns (e.g: 'Close'), because the technical indicators will need to use these values
# I Will not be performing log transformation to handle outliers for columns: Daily_Return and Volume
# -> Daily_Return (measured as % change in stock's price) can be both +ve and -ve and extreme returns can occur, log transformation not required
# -> Although volume data can have outliers, it will be uesd as it is because unlike price data, it does not have a natural logarithmic interpretation. In fact, outliers in volume may carry valuable information about market events and trading behaviour

# I will be performing log transformation to handle outliers in columns: Adj. CLose, Close, High, Low, Open
# -> These columns are often right-skewed due to presence of extreme price values (stock splits, mergers, market events)
# -> Applying log. transformation to these columns can mitigate impact of outliers and make data more normally distributed

In [ ]:
# Using histogram to check data distribution if they are right-skewed (mean>median)

# For Adj Close columns
columns_to_plot = ["AAPL_Adj Close", "NVDA_Adj Close", "MAR_Adj Close", "LMT_Adj Close", "BA_Adj Close"]
for column in columns_to_plot:
    plt.hist(merged[column], bins=30, alpha=0.6, label=column)

# Add labels and title
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histograms of AdjClose Columns')
plt.legend(loc='upper right')

# Show the plot
plt.show()


# Identify outliers using Z-score and handle them using log transformation
# Set a common threshold for identifying outliers
threshold = 3  # indicates outliers are data points 3 standard deviations away from the mean

for column_name in columns_to_plot:
    # Calculate the Z-score for the current column
    z_scores = np.abs(stats.zscore(merged[column_name]))

    # Identify outliers based on the Z-score
    outliers = merged[column_name][z_scores > threshold]

    # Apply log transformation to outliers and replace in the DataFrame
    merged.loc[z_scores > threshold, column_name] = np.log1p(outliers)

    # Check the distribution after log transformation
    sns.histplot(merged[column_name], kde=True)
    plt.xlabel(column_name)
    plt.title(f'Distribution of {column_name} after log transformation')
    plt.show()

In [ ]:
# Using histogram to check data distribution if they are right-skewed (mean>median)

# For Close columns
columns_to_plot = ["AAPL_Close", "NVDA_Close", "MAR_Close", "LMT_Close", "BA_Close"]
for column in columns_to_plot:
    plt.hist(merged[column], bins=30, alpha=0.6, label=column)

# Add labels and title
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histograms of Close Columns')
plt.legend(loc='upper right')

# Show the plot
plt.show()


# Identify outliers using Z-score and handle them using log transformation
# Set a common threshold for identifying outliers
threshold = 3  # indicates outliers are data points 3 standard deviations away from the mean

for column_name in columns_to_plot:
    # Calculate the Z-score for the current column
    z_scores = np.abs(stats.zscore(merged[column_name]))

    # Identify outliers based on the Z-score
    outliers = merged[column_name][z_scores > threshold]

    # Apply log transformation to outliers and replace in the DataFrame
    merged.loc[z_scores > threshold, column_name] = np.log1p(outliers)

    # Check the distribution after log transformation
    sns.histplot(merged[column_name], kde=True)
    plt.xlabel(column_name)
    plt.title(f'Distribution of {column_name} after log transformation')
    plt.show()

In [ ]:
# Using histogram to check data distribution if they are right-skewed (mean>median)

# For High columns
columns_to_plot = ["AAPL_High", "NVDA_High", "MAR_High", "LMT_High", "BA_High"]
for column in columns_to_plot:
    plt.hist(merged[column], bins=30, alpha=0.6, label=column)

# Add labels and title
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histograms of High Columns')
plt.legend(loc='upper right')

# Show the plot
plt.show()


# Identify outliers using Z-score and handle them using log transformation
# Set a common threshold for identifying outliers
threshold = 3  # indicates outliers are data points 3 standard deviations away from the mean

for column_name in columns_to_plot:
    # Calculate the Z-score for the current column
    z_scores = np.abs(stats.zscore(merged[column_name]))

    # Identify outliers based on the Z-score
    outliers = merged[column_name][z_scores > threshold]

    # Apply log transformation to outliers and replace in the DataFrame
    merged.loc[z_scores > threshold, column_name] = np.log1p(outliers)

    # Check the distribution after log transformation
    sns.histplot(merged[column_name], kde=True)
    plt.xlabel(column_name)
    plt.title(f'Distribution of {column_name} after log transformation')
    plt.show()

In [ ]:
# Using histogram to check data distribution if they are right-skewed (mean>median)

# For Low columns
columns_to_plot = ["AAPL_Low", "NVDA_Low", "MAR_Low", "LMT_Low", "BA_Low"]
for column in columns_to_plot:
    plt.hist(merged[column], bins=30, alpha=0.6, label=column)

# Add labels and title
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histograms of Low Columns')
plt.legend(loc='upper right')

# Show the plot
plt.show()


# Identify outliers using Z-score and handle them using log transformation
# Set a common threshold for identifying outliers
threshold = 3  # indicates outliers are data points 3 standard deviations away from the mean

for column_name in columns_to_plot:
    # Calculate the Z-score for the current column
    z_scores = np.abs(stats.zscore(merged[column_name]))

    # Identify outliers based on the Z-score
    outliers = merged[column_name][z_scores > threshold]

    # Apply log transformation to outliers and replace in the DataFrame
    merged.loc[z_scores > threshold, column_name] = np.log1p(outliers)

    # Check the distribution after log transformation
    sns.histplot(merged[column_name], kde=True)
    plt.xlabel(column_name)
    plt.title(f'Distribution of {column_name} after log transformation')
    plt.show()

In [ ]:
# Using histogram to check data distribution if they are right-skewed (mean>median)

# For High columns
columns_to_plot = ["AAPL_Open", "NVDA_Open", "MAR_Open", "LMT_Open", "BA_Open"]
for column in columns_to_plot:
    plt.hist(merged[column], bins=30, alpha=0.6, label=column)

# Add labels and title
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histograms of Open Columns')
plt.legend(loc='upper right')

# Show the plot
plt.show()


# Identify outliers using Z-score and handle them using log transformation
# Set a common threshold for identifying outliers
threshold = 3  # indicates outliers are data points 3 standard deviations away from the mean

for column_name in columns_to_plot:
    # Calculate the Z-score for the current column
    z_scores = np.abs(stats.zscore(merged[column_name]))

    # Identify outliers based on the Z-score
    outliers = merged[column_name][z_scores > threshold]

    # Apply log transformation to outliers and replace in the DataFrame
    merged.loc[z_scores > threshold, column_name] = np.log1p(outliers)

    # Check the distribution after log transformation
    sns.histplot(merged[column_name], kde=True)
    plt.xlabel(column_name)
    plt.title(f'Distribution of {column_name} after log transformation')
    plt.show()

In [ ]:
merged        # merged dataframe outliers will now be replaced

In [ ]:
# Check for NaN values in any of the columns (except sentiment_score)
# 1st NaN row: first row for Daily_Return  (because there's no return on 'first day')
# 2nd NaN row: I have found out that the NaN value comes from the row 2356 in excel => which is due to: an earnings call by MAR was done on a Saturday (2019-05-11)

# Will be using zero fill to replace (Assume no return on first day) 1st NaN row for "Daily_Return" columns, Using Median to replace 1st NaN row of other columns,

# Check for NaN values in all columns of the DataFrame and count them
nan_counts = merged.isna().sum()

# Display columns with NaN values and the count of NaN values
columns_with_nan = nan_counts[nan_counts > 0]
print(columns_with_nan)


zero_fill_columns = ["AAPL_Daily_Return", "NVDA_Daily_Return", "MAR_Daily_Return", "LMT_Daily_Return", "BA_Daily_Return"]
for column in zero_fill_columns:
    if merged[column].isna().any():
        first_nan_index = merged[column].index[merged[column].isna()].tolist()[0]         # Only for the first row of Daily_Return columns

        merged.at[first_nan_index, column] = 0


# Display columns with NaN values and the count of NaN values
nan_counts = merged.isna().sum()
columns_with_nan = nan_counts[nan_counts > 0]
print(columns_with_nan)

columns_to_replace_outliers = ["AAPL_Adj Close", "AAPL_Close", "AAPL_Daily_Return", "AAPL_High", "AAPL_Low", "AAPL_Open", "AAPL_Volume",
                               "NVDA_Adj Close", "NVDA_Close", "NVDA_Daily_Return", "NVDA_High", "NVDA_Low", "NVDA_Open", "NVDA_Volume",
                               "MAR_Adj Close", "MAR_Close", "MAR_Daily_Return", "MAR_High", "MAR_Low", "MAR_Open", "MAR_Volume",
                               "LMT_Adj Close", "LMT_Close", "LMT_Daily_Return", "LMT_High", "LMT_Low", "LMT_Open", "LMT_Volume",
                               "BA_Adj Close", "BA_Close", "BA_Daily_Return", "BA_High", "BA_Low", "BA_Open", "BA_Volume",
                               ]


# Function that replaces NaN values of column with median
def replace_nan_with_median(df, columns_to_process):
    for column in columns_to_process:
        median = df[column].median()
        df[column].fillna(median, inplace=True)
    return df

merged = replace_nan_with_median(merged, columns_to_replace_outliers)

print(merged)


In [ ]:
# Interpolate the sentiment scores for all 5 stocks for those non earnings call dates using linear interpolation
columns_to_interpolate = ['AAPL_Sentiment_score', 'NVDA_Sentiment_score', 'MAR_Sentiment_score', 'LMT_Sentiment_score', 'BA_Sentiment_score']  # Replace with your column names
for column in columns_to_interpolate:
    merged[column].interpolate(method='linear', inplace=True)

merged.info()

In [ ]:
#Display columns with NaN values and the count of NaN values
nan_counts = merged.isna().sum()
columns_with_nan = nan_counts[nan_counts > 0]
print(columns_with_nan)

# Since there are still leftover NaN for sentiment score column, possibly due to presence of consecutive NaN values, I will use backfill to handle them
columns_to_backfill = ['AAPL_Sentiment_score', 'NVDA_Sentiment_score', 'MAR_Sentiment_score', 'LMT_Sentiment_score', 'BA_Sentiment_score']
for column in columns_to_backfill:
    merged[column].fillna(method='bfill', inplace=True)

merged.tail(50)

# Confirm there is no leftover NaN values
nan_counts_new = merged.isna().sum()
columns_with_nan_new = nan_counts_new[nan_counts > 0]
print(columns_with_nan_new)

In [ ]:
merged

In [ ]:
# Add additional features: Technical indicators to measure price momentum

portfolio_stocks = ["AAPL", "NVDA", "MAR", "LMT", "BA"]

def add_technical_indicators(df, stock_symbol):
    # Calculate historical volatility
    df[f'{stock_symbol}_Volatility'] = df[f'{stock_symbol}_Close'].rolling(window=21).std()

    # Calculate Simple Moving Average (SMA)
    df[f'{stock_symbol}_SMA'] = df[f'{stock_symbol}_Close'].rolling(window=14).mean()

    # Calculate Upper Bollinger Band
    df[f'{stock_symbol}_Upper_Bollinger'] = df[f'{stock_symbol}_SMA'] + 2 * df[f'{stock_symbol}_Volatility']

    # Calculate Lower Bollinger Band
    df[f'{stock_symbol}_Lower_Bollinger'] = df[f'{stock_symbol}_SMA'] - 2 * df[f'{stock_symbol}_Volatility']

    # Calculate Moving Average Convergence Divergence (MACD)
    df[f'{stock_symbol}_MACD'] = df[f'{stock_symbol}_Close'].rolling(window=12).mean() - df[f'{stock_symbol}_Close'].rolling(window=26).mean()

    # Calculate Relative Strength Index (RSI)
    delta = df[f'{stock_symbol}_Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    df[f'{stock_symbol}_RSI'] = rsi

for each_stock in portfolio_stocks:
    add_technical_indicators(merged, each_stock)

merged

In [ ]:
# Check NaN counts for the additional technical indicators columns
# Display columns with NaN values and the count of NaN values
nan_counts = merged.isna().sum()
columns_with_nan = nan_counts[nan_counts > 0]
print(columns_with_nan)

In [ ]:
# Using backfill interpolation to fill up the NaN values for the above columns, useful for consecutive NaN values
nan_technical_indicators = merged.isna()
for column in nan_technical_indicators:
    merged[column].interpolate(method='bfill', inplace=True)

# Check if there are still NaN values
new_nan_counts = merged.isna().sum()
new_columns_with_nan = new_nan_counts[new_nan_counts > 0]
print(new_columns_with_nan)
print('\n')

# Confrim no leftover NaN values for all columns in dataframe
print(merged.isna().sum())

In [ ]:
merged.sort_index(axis=1, inplace=True)
merged.set_index("Date", inplace = True)
merged.to_excel('merged.xlsx', index=True)
# merged

In [ ]:
# Standardize features so that they have mean of 0 and std of 1, ensure all features are on a similar scale
# Define the columns you want to standardize (excluding non-numeric columns)
columns_to_standardize = merged.select_dtypes(include=[np.number]).columns.tolist()

# Create a StandardScaler instance
scaler = StandardScaler()

# Standardize the selected columns
merged[columns_to_standardize] = scaler.fit_transform(merged[columns_to_standardize])

In [ ]:
merged

#Performing Data Visualization to provide context for feature selection process

In [ ]:
# See all columns in dataframe
merged.columns.values

In [ ]:
### TO BE EDITED: After dataframe is back to multiindex, then can use plotly.express to plot interactive scatter matrix to see relationship of columns for each stock

# Select a subset of columns for visualization => Correlation Analysis
# selected_columns = [
#     'AAPL_Adj Close', 'AAPL_Close', 'AAPL_Daily_Return', 'AAPL_High', 'AAPL_Low', 'AAPL_Open',
#     'NVDA_Adj Close', 'NVDA_Close', 'NVDA_Daily_Return', 'NVDA_High', 'NVDA_Low', 'NVDA_Open',
#     'MAR_Adj Close', 'MAR_Close', 'MAR_Daily_Return', 'MAR_High', 'MAR_Low', 'MAR_Open'
# ]

# Create an interactive scatter matrix


# fig = px.scatter_matrix(merged, dimensions=selected_columns, color="AAPL_Adj Close", title="Stock Data Relationships")
# fig.update_traces(marker=dict(size=3), selector=dict(mode='markers'))
# fig.show()

# Feature Selection using xgboost

In [ ]:
# Using xgboost model for feature selection, to select most important features for LSTM Model training
feature_columns_AAPL = ['AAPL_Adj Close', 'AAPL_Close', 'AAPL_Daily_Return',
       'AAPL_High', 'AAPL_Low', 'AAPL_Open', 'AAPL_Volume',
       'AAPL_Sentiment_score', 'AAPL_Volatility', 'AAPL_SMA',
       'AAPL_Upper_Bollinger', 'AAPL_Lower_Bollinger', 'AAPL_MACD',
       'AAPL_RSI']

feature_columns_NVDA = ['NVDA_Adj Close', 'NVDA_Close',
       'NVDA_Daily_Return', 'NVDA_High', 'NVDA_Low', 'NVDA_Open',
       'NVDA_Volume', 'NVDA_Sentiment_score', 'NVDA_Volatility', 'NVDA_SMA', 'NVDA_Upper_Bollinger',
       'NVDA_Lower_Bollinger', 'NVDA_MACD', 'NVDA_RSI']


feature_columns_MAR = ['MAR_Adj Close',
       'MAR_Close', 'MAR_Daily_Return', 'MAR_High', 'MAR_Low', 'MAR_Open',
       'MAR_Volume', 'MAR_Sentiment_score', 'MAR_Volatility',
       'MAR_SMA', 'MAR_Upper_Bollinger', 'MAR_Lower_Bollinger',
       'MAR_MACD', 'MAR_RSI']


feature_columns_LMT = [ 'LMT_Adj Close', 'LMT_Close',
       'LMT_Daily_Return', 'LMT_High', 'LMT_Low', 'LMT_Open',
       'LMT_Volume', 'LMT_Sentiment_score',  'LMT_Volatility', 'LMT_SMA',
       'LMT_Upper_Bollinger', 'LMT_Lower_Bollinger', 'LMT_MACD',
       'LMT_RSI']


feature_columns_BA = ['BA_Adj Close', 'BA_Close',
       'BA_Daily_Return', 'BA_High', 'BA_Low', 'BA_Open', 'BA_Volume',
       'BA_Sentiment_score', 'BA_Volatility', 'BA_SMA', 'BA_Upper_Bollinger',
       'BA_Lower_Bollinger', 'BA_MACD', 'BA_RSI']


In [ ]:
# For AAPL
X = merged[feature_columns_AAPL]

# Train an XGBoost Regressor
model = XGBRegressor()
model.fit(X, X)  # Use the same features for both X and y => Unsupervised feature selection

# Plot feature importances
fig, ax = plt.subplots(figsize=(10, 6))
plot_importance(model, ax=ax)

# Adjust font size and spacing for y-axis labels
plt.yticks(fontsize=8)
plt.tight_layout()  # Prevent labels from being cut off

plt.show()

In [ ]:
# For NVDA
X = merged[feature_columns_NVDA]

# Train an XGBoost Regressor
model = XGBRegressor()
model.fit(X, X)  # Use the same features for both X and y => Unsupervised feature selection

# Plot feature importances
fig, ax = plt.subplots(figsize=(10, 6))
plot_importance(model, ax=ax)

# Adjust font size and spacing for y-axis labels
plt.yticks(fontsize=8)
plt.tight_layout()  # Prevent labels from being cut off

plt.show()

In [ ]:
# For MAR
X = merged[feature_columns_MAR]

# Train an XGBoost Regressor
model = XGBRegressor()
model.fit(X, X)  # Use the same features for both X and y => Unsupervised feature selection

# Plot feature importances
fig, ax = plt.subplots(figsize=(10, 6))
plot_importance(model, ax=ax)

# Adjust font size and spacing for y-axis labels
plt.yticks(fontsize=8)
plt.tight_layout()  # Prevent labels from being cut off

plt.show()

In [ ]:
# For LMT
X = merged[feature_columns_LMT]

# Train an XGBoost Regressor
model = XGBRegressor()
model.fit(X, X)  # Use the same features for both X and y => Unsupervised feature selection

# Plot feature importances
fig, ax = plt.subplots(figsize=(10, 6))
plot_importance(model, ax=ax)

# Adjust font size and spacing for y-axis labels
plt.yticks(fontsize=8)
plt.tight_layout()  # Prevent labels from being cut off

plt.show()

In [ ]:
# For BA
X = merged[feature_columns_BA]

# Train an XGBoost Regressor
model = XGBRegressor()
model.fit(X, X)  # Use the same features for both X and y => Unsupervised feature selection

# Plot feature importances
fig, ax = plt.subplots(figsize=(10, 6))
plot_importance(model, ax=ax)

# Adjust font size and spacing for y-axis labels
plt.yticks(fontsize=8)
plt.tight_layout()  # Prevent labels from being cut off

plt.show()

In [ ]:
# LSTM Model